# Udacity Data Wrangling Project
###### Suraag Gupta

#### Map Area: Columbus, OH USA
The OSM dataset I chose was for Columbus, OH simply because it is where I live.
## Processing the Dataset
I started off with importing the libraries I knew I would be needing and the OSM file.

In [1]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from pymongo import MongoClient
import pprint, re, json, codecs
from OSMAudit import *
from OSMProcess import *
from OSMShape import *

OSM_FILE="ColumbusOH.osm"

### Step 1: Preliminary Information
Then I began running all the functions that I had worked on in the CaseStudy lessons.
I used the count_tags, key_type, process_users, & get_address_types functions to get a sense of the data I was looking at and what I would need to do moving forward.

Thankfully, there were no problems in running this information, so I managed to move on fairly quickly.

### Step 2: Auditing
Looking at the different addr types from the output of the previous function, I decided that I should go through the city, country, county, postcode, & state keys alongside the street to ensure that all the values were uniform and as expected.

In [2]:
with open(OSM_FILE, 'r') as columbus:
    audit_street_names(columbus)

{'101': set(['E Dublin Granville Rd #101']),
 '105': set(['Mill St Suite 105']),
 '250': set(['Chambers Rd #250']),
 '2D': set(['Copeland Mill Rd #2D']),
 '320': set(['W Old Wilson Bridge Rd #320']),
 '762': set(['State Route 762']),
 '8877': set(['8877']),
 '8897': set(['8897']),
 '8906': set(['8906']),
 'Ave': set(['N Cassady Ave']),
 'Blvd': set(['Channingway Blvd']),
 'Broadway': set(['Broadway']),
 'Center': set(['Easton Town Center', 'Mc Naughten Center', 'Tremont Center']),
 'Circle': set(['Ellington Circle', 'Gateway Circle', 'Parkcenter Circle']),
 'Crossing': set(['Morse Crossing']),
 'Dr': set(['Auto Mall Dr',
            'Green Meadows Dr',
            'Humphries Dr',
            'Rentra Dr',
            'Wake Dr',
            'Westpoint Plaza Dr',
            'Westpointe Plaza Dr']),
 'Dr.': set(['Centerpoint Dr.', 'Christopher John Dr.', 'Mark Andrew Dr.']),
 'East': set(['Byers Circle East',
              'Easton Loop East',
              'New Albany Road East',
        

After running my audits, I created the appropriate mapping dicts for each of the addr types.

The cities just had a few capitalizations and extra state notations to get rid of, there was only one entry for counties, I plan to ignore and delete the Alabama & Georgia elements as they should not be here, I'm going to set all the country codes to the 3 letter version, and there were some zip codes that needed to be more uniform.

Once that was done, the last step was to shape the elements into the specified format and output it to a json file.

In [3]:
columbusData=process_map(OSM_FILE)

## Exploring the Database
### Step 3: Creating and Adding the Dataset to MongoDB
Now that the cleaning process is done, I can go ahead and add the json file to a database.

In [4]:
client = MongoClient('localhost:27017')
db = client.udacity_db
db.columbusOSM.delete_many({})
db.columbusOSM.insert_many(columbusData);

### Step 4: Querying the Database
Now all that's left is to query the database for various information. I'll start with the size of the database and how many documents there are.

In [5]:
aggregate=[{'$collStats':{'storageStats':{}}}]
for a in db.columbusOSM.aggregate(aggregate):
    print a['storageStats']['size']

180387893


In [6]:
db.columbusOSM.find().count()

805665

That's over 800,000 documents in about 172 MB of data. Next up is how many unique users have contributed to this. I already checked the number earlier, so it should be similar.

In [7]:
len(db.columbusOSM.find().distinct('created.user'))

856

That's 10 less than my original number, so it seems that some users weren't part of the final dataset that got added in. Next, I'll query to find the number of nodes and ways.

In [8]:
aggregate=[{'$group':{'_id':'$type', 'count':{'$sum':1}}}]
for a in db.columbusOSM.aggregate(aggregate):
    print a

{u'count': 17, u'_id': u'multipolygon'}
{u'count': 84846, u'_id': u'way'}
{u'count': 4, u'_id': u'multi-storey'}
{u'count': 720798, u'_id': u'node'}


Relating this to my original tag count of the XML file, all but 1 of the node & way tags made it into the final database.

Next I'll take a look at what the most common amenities are.

In [9]:
aggregate=[{'$group':{'_id':'$amenity', 'count':{'$sum':1}}},
           {'$sort':{'count':-1}},
           {'$limit':10}]
for a in db.columbusOSM.aggregate(aggregate):
    print a

{u'count': 799423, u'_id': None}
{u'count': 2566, u'_id': u'parking'}
{u'count': 741, u'_id': u'place_of_worship'}
{u'count': 550, u'_id': u'restaurant'}
{u'count': 505, u'_id': u'school'}
{u'count': 261, u'_id': u'fast_food'}
{u'count': 141, u'_id': u'fuel'}
{u'count': 110, u'_id': u'bench'}
{u'count': 98, u'_id': u'bank'}
{u'count': 94, u'_id': u'grave_yard'}


Now, earlier when I ran my state audits, I saw that there were entries for Alabama & Georgia in the dataset. I'm going to look for those entries now that they're a little cleaner to look at and determine whether or not they belong.

In [10]:
query={'address.state':{'$exists':1, '$ne':'OH'}}
for q in db.columbusOSM.find(query):
    pprint.pprint(q)

{u'_id': ObjectId('5a01e94fa481d92628dc7689'),
 u'address': {u'city': u'Adamsville',
              u'housenumber': u'35005',
              u'postcode': u'35005',
              u'state': u'Alabama',
              u'street': u'East Broad Street'},
 u'amenity': u'fuel',
 u'created': {u'changeset': u'52339826',
              u'timestamp': u'2017-09-24T21:28:47Z',
              u'uid': u'339581',
              u'user': u'nyuriks',
              u'version': u'4'},
 u'id': u'1278853280',
 u'name': u'Marathon',
 u'opening_hours': u'sasqsq',
 u'operator': u'sqsqsqsqsqs',
 u'pos': [39.9846503, -82.8153421],
 u'type': u'node',
 u'wikidata': u'Q4590796',
 u'wikipedia': u'en:1995 World Marathon Cup'}
{u'_id': ObjectId('5a01e958a481d92628e1ded1'),
 u'address': {u'city': u'Columbus',
              u'housenumber': u'1773',
              u'postcode': u'43212',
              u'state': u'ga',
              u'street': u'West Fifth Avenue'},
 u'amenity': u'restaurant',
 u'created': {u'changeset': u'4588549

The first document is definitely not in Columbus, OH. So I'll delete that.

In [11]:
query={'address.state':'Alabama'}
db.columbusOSM.delete_one(query);

The second document however, seems to have just made a mistake on the state, as the name, address, city, & postcode are all correct, so for this one, I'll simply update the state.

In [12]:
query={'address.state':'ga'}
peking=db.columbusOSM.find_one(query)
peking['address']['state']='OH'
db.columbusOSM.replace_one(query, peking);

There were also some invalid postcodes in the data, but getting the correct values for those would be a little tedious, so instead of updating those, I'm just going to remove the attributes for those.

In [13]:
query={'$nor':[{'address.postcode':{'$exists':0}},
               {'address.postcode':{'$regex':'43[0-9]{3}'}}]}
db.columbusOSM.update_many(query, {'$unset':{'address.postcode':''}})

aggregate=[{'$group':{'_id':'$address.postcode', 'count':{'$sum':1}}},
           {'$sort':{'count':-1}},
           {'$limit':10}]
for a in db.columbusOSM.aggregate(aggregate):
    print a

{u'count': 803891, u'_id': None}
{u'count': 333, u'_id': u'43017'}
{u'count': 218, u'_id': u'43235'}
{u'count': 111, u'_id': u'43221'}
{u'count': 97, u'_id': u'43215'}
{u'count': 90, u'_id': u'43210'}
{u'count': 87, u'_id': u'43026'}
{u'count': 78, u'_id': u'43212'}
{u'count': 72, u'_id': u'43201'}
{u'count': 60, u'_id': u'43202'}


Just for my curiosity, I'm going to see which city has the most information in it.

In [14]:
aggregate=[{'$group':{'_id':'$address.city', 'count':{'$sum':1}}},
           {'$sort':{'count':-1}},
           {'$limit':10}]
for a in db.columbusOSM.aggregate(aggregate):
    print a

{u'count': 803790, u'_id': None}
{u'count': 998, u'_id': u'Columbus'}
{u'count': 345, u'_id': u'Dublin'}
{u'count': 98, u'_id': u'Upper Arlington'}
{u'count': 85, u'_id': u'Hilliard'}
{u'count': 58, u'_id': u'Pickerington'}
{u'count': 37, u'_id': u'Powell'}
{u'count': 34, u'_id': u'Lewis Center'}
{u'count': 30, u'_id': u'Worthington'}
{u'count': 30, u'_id': u'Galloway'}


As expected, the main city of Columbus has more documents rather than any of its suburbs.
### Additional Improvements
After going through some of the documents, one of the things that bothered me the most was the lack of complete address information. While many had no information at all, several were only partially complete.

In [15]:
query={'address':{'$exists':1}}
print db.columbusOSM.find(query).count()

2771


Only about 3000 documents in over 800,000 documents had some address information, with even less having it fully completed.

One possible solution for this would be to figure out the latitude and longitude ranges in which certain geo-specific attributes fall under, like post codes, cities/districts, counties, and then states/countries on a larger scale.

This solution would be extremely tedious to do manually, as it would require setting boundaries for the millions of combinations of the different address notations around the world, but it seems like something that could be possible with machine learning algorithms. Once this algorithm was implemented, it could even be refined to the degree that it is able to fill in street names on its own.

## Conclusion
I enjoyed this project. It gave me a lot of practice with several Python libraries and helped me to understand more of what Data Science & Analysis is all about.

For this dataset in particular, there are certainly areas that need improvement, mostly concerning missing data and validity of it, but I think it has been cleaned fairly well to be sufficiently readable.